In [1]:
# Dependencies
import requests
import json
import pandas as pd
from pprint import pprint
from config import api_key, username, password, ipaddress, port, dbname
from sqlalchemy import create_engine


In [2]:
# Specify the URL
elections_url = "https://api.open.fec.gov/v1/elections/?"

In [3]:
# Create districts list
districts=[]

# Loop through integers to append into the list
for n in range(1, 54):
    i=str(n).zfill(2)
    districts.append(i)
    n+=1
    
# Print the list
print(districts)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53']


In [6]:
# Set params in URL
sort_null_only="true"
office="house"
state="CA"
per_page="34"
sort_nulls_last="true"
sort_hide_null="true"
cycle="2016"
sort="-total_receipts"
hide_null="true"
page=1
election_full="true"

In [7]:
# Create an empty dataframe
combined_df=pd.DataFrame()

# Loop through all districts ID in districts
for district in districts:
    
    # Create empty lists
    committee_ids=[]
    cash_on_hand_end_period=[]
    candidate_pcc_id=[]
    total_receipts=[]
    coverage_end_date=[]
    candidate_election_year=[]
    candidate_name=[]
    total_disbursements=[]
    party_full=[]
    candidate_id=[]
    
    # Define params
    params= {
        "sort_null_only":sort_null_only,
        "office":office,
        "state":state,
        "per_page":per_page,
        "sort_nulls_last":sort_nulls_last,
        "sort_hide_null":sort_hide_null,
        "cycle":cycle,
        "sort":sort,
        "hide_null":hide_null,
        "page":page,
        "election_full":election_full,
        "api_key":api_key,
        "district":district
    }

    # Run request in JSON
    response=requests.get(elections_url,params=params).json()
    
    # Set variable to represent only results from the API
    results=response["results"]
    
    # Loop through every result in results
    for result in results:

        # Try to add information from results to the empty list
        try:
            committee_ids.append(result["committee_ids"][0])
            cash_on_hand_end_period.append(result["cash_on_hand_end_period"])
            candidate_pcc_id.append(result["candidate_pcc_id"])
            total_receipts.append(result["total_receipts"])
            coverage_end_date.append(result["coverage_end_date"])
            candidate_election_year.append(result["candidate_election_year"])
            candidate_name.append(result["candidate_name"])
            total_disbursements.append(result["total_disbursements"])
            party_full.append(result["party_full"])
            candidate_id.append(result["candidate_id"])

        # Otherwise return none
        except:
            None

    # Create a dataframe to show all results
    election_df = pd.DataFrame({
        "Committee ID": committee_ids,
        "Cash on Hand End Period": cash_on_hand_end_period,
        "Candidate PCC ID":candidate_pcc_id,
        "Total Receipts": total_receipts,
        "End Date": coverage_end_date,
        "Election Year": candidate_election_year,
        "Candidate Name": candidate_name,
        "Total Disbursements": total_disbursements,
        "Party": party_full,
        "Candidate ID": candidate_id,
        "district":district
    })
    
    # Concat each election_df for one district into a combined dataframe with all districts
    combined_df=pd.concat([combined_df, election_df], ignore_index=True)

In [8]:
# Print the dataframe
combined_df

,Committee ID,Cash on Hand End Period,Candidate PCC ID,Total Receipts,End Date,Election Year,Candidate Name,Total Disbursements,Party,Candidate ID,district
0,C00509422,89475.60,C00509422,800277.02,2016-12-31T00:00:00,2016,"LAMALFA, DOUG",808869.52,REPUBLICAN PARTY,H2CA02142,01
1,C00608265,0.00,C00608265,183682.00,2016-12-31T00:00:00,2016,"MONTES, JOSEPH",183682.00,REPUBLICAN PARTY,H6CA01186,01
2,C00609958,3592.42,C00609958,138508.99,2016-12-31T00:00:00,2016,"REED, JAMES E",134916.57,DEMOCRATIC PARTY,H6CA01194,01
3,C00605535,0.00,C00605535,27709.03,2016-05-10T00:00:00,2016,"WRIGHT, DOUGLAS A.",27709.03,REPUBLICAN PARTY,H6CA01178,01
4,C00615104,0.00,C00615104,0.00,None,2016,"OXLEY, GARY ALLEN",0.00,REPUBLICAN PARTY,H2CA01151,01
...,...,...,...,...,...,...,...,...,...,...,...
179,C00546861,0.00,C00610576,517.93,2015-04-05T00:00:00,2016,"SIMON, FRED J JR MD",790.44,REPUBLICAN PARTY,H4CA52077,52
180,C00344671,255984.17,C00344671,485051.35,2016-12-31T00:00:00,2016,"DAVIS, SUSAN",470870.68,DEMOCRATIC PARTY,H0CA49055,53
181,C00575860,1646.88,C00575860,122886.07,2016-12-31T00:00:00,2016,"VELTMEYER, JAMES",121239.19,REPUBLICAN PARTY,H6CA53054,53
182,C00573915,0.00,C00573915,35278.00,2016-07-11T00:00:00,2016,"ASH, JAMES",35278.00,REPUBLICAN PARTY,H6CA53047,53


In [9]:
# Create engine and connect it to Postgresql
engine=create_engine(f"postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}")
conn=engine.connect()

In [11]:
# Create table name and export dataframe to Postgresql
table_name = 'election_table'
combined_df.to_sql(table_name, conn)